## Introduction

This tutorial demonstrates various **pair-wise point cloud registration** methods implemented in Open3D.


The registration methods can be roughly grouped into two categories.

1. Global Registration

The **global registration** is a family of registration methods that aims to recover the relative 3D rigid transformation between two possibily overlapping point clouds.  
They estimate the rough alignment of input point clouds without requiring the initial alignment information.  
There are two available global registration methods implemented in Open3D: **RANSAC** and **FGR**.  

2. Local Registration

Unlike global registration methods, the **local registration methods** process roughly aligned point clouds and outputs refined transformation that tightly align two point clouds.  
Iterative Closest Points (ICP) is the one of the most representative local registration methods.  
In Open3D, two variants of ICP is provided: 1. point-to-point ICP and 2. point-to-plane ICP.  


In this tutorial, we demonstrate both global registration (RANSAC, FGR) and local registration (ICP) on real-world indoor RGB-D datasets using Open3D.

In [ ]:
import open3d as o3d
import numpy as np
import copy
import time


## 1. Dataset preparation

The first step in this tutorial is to download datasets and preprocess them.  
In Open3D, there is a helper function to download various demo datasets for various tasks.  

For global registration methods, we need to extract local geometric features from point cloud data to perform feature matching.
In this tutorial, we compute FPFH feature for each point.  
The FPFH feature is a 33-dimensional vector that describes the local geometric property of a point. 

In [ ]:
def preprocess_pcd(pcd, voxel_size):
    pcd_down = pcd.voxel_down_sample(voxel_size)

    pcd_down.estimate_normals(
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 2.0, max_nn=30)
    )
    feature = o3d.pipelines.registration.compute_fpfh_feature(
        pcd_down,
        o3d.geometry.KDTreeSearchParamHybrid(radius=voxel_size * 5.0, max_nn=100),
    )
    return pcd_down, feature

In [ ]:
def prepare_dataset(voxel_size):
    pcds = o3d.data.DemoICPPointClouds()
    src_pcd = o3d.io.read_point_cloud(pcds.paths[0])
    tgt_pcd = o3d.io.read_point_cloud(pcds.paths[1])

    src_down, src_feature = preprocess_pcd(src_pcd, voxel_size)
    tgt_down, tgt_feature = preprocess_pcd(tgt_pcd, voxel_size)
    return src_down, src_feature, tgt_down, tgt_feature

The below function visualize registration results using Open3D.

In [ ]:
def draw_registration_results(src, tgt, trans, title=None):
    src_copy = copy.deepcopy(src)
    tgt_copy = copy.deepcopy(tgt)

    src_copy.paint_uniform_color([1, 0.706, 0])
    tgt_copy.paint_uniform_color([0, 0.651, 0.929])

    src_copy.transform(trans)
    o3d.visualization.draw_geometries(
        [src_copy, tgt_copy],
        window_name=title,
        zoom=0.8,
        front=[0.6452, -0.3036, -0.7011],
        lookat=[1.9892, 2.0208, 1.8945],
        up=[-0.2779, -0.9482, 0.1556],
    )

This code below visualizes the two point clouds with their initial aligment.

In [ ]:
VOXEL_SIZE = 0.05  # 2cm
src_pcd, src_feature, tgt_pcd, tgt_feature = prepare_dataset(VOXEL_SIZE)
draw_registration_results(src_pcd, tgt_pcd, np.eye(4), title="before registration")

## 2. RANSAC

First, we use RANSAC for global registration. 
RANSAC takes two point clouds and corresponding FPFH features and estimate the rigid transformation parameters by repeating multiple iterations of *hypothesis-then-verify* loop.
In Open3D, the function ```registration_ransac_based_on_feature_matching``` is implemented in the ```o3d.pipelines.registration``` namespace.
There are various arguments we need to set up. Please check the Open3D's documentation for more details. 

In [ ]:
def global_registration_by_ransac(
    src_pcd, src_feature, tgt_pcd, tgt_feature, voxel_size
):
    ts = time.time()
    result = o3d.pipelines.registration.registration_ransac_based_on_feature_matching(
        src_pcd,
        tgt_pcd,
        src_feature,
        tgt_feature,
        mutual_filter=True,
        max_correspondence_distance=voxel_size * 1.5,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPoint(
            False
        ),
        ransac_n=3,
        checkers=[
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnEdgeLength(0.9),
            o3d.pipelines.registration.CorrespondenceCheckerBasedOnDistance(
                voxel_size * 1.5
            ),
        ],
        criteria=o3d.pipelines.registration.RANSACConvergenceCriteria(100000, 0.999),
    )
    print(f"[RANSAC] takes {time.time() - ts:.3f} (s)")
    return result

The below code visualizes the results of RANSAC. As can be seen in the visualization, RANSAC successfully align input point clouds.

In [ ]:
result_ransac = global_registration_by_ransac(
    src_pcd, src_feature, tgt_pcd, tgt_feature, VOXEL_SIZE
)
draw_registration_results(
    src_pcd, tgt_pcd, result_ransac.transformation, "after global registration [RANSAC]"
)

## 3. FGR

Although RANSAC has been chosen as the de facto standard global registration method for long time, it may takes a long time due to the countless proposals and evaluations.
FGR proposes a faster approach by eliminating repetitive proposal and evaluations and quickly optimizes the line process weights of few correspondences.

The below function run the FGR implmented in ```registration_fgr_based_on_feature_matching``` function.

In [ ]:
def global_registration_by_fgr(src_pcd, src_feature, tgt_pcd, tgt_feature, voxel_size):
    ts = time.time()
    result = o3d.pipelines.registration.registration_fgr_based_on_feature_matching(
        src_pcd,
        tgt_pcd,
        src_feature,
        tgt_feature,
        o3d.pipelines.registration.FastGlobalRegistrationOption(
            maximum_correspondence_distance=voxel_size * 0.5
        ),
    )
    print(f"[FGR] takes {time.time() - ts:.3f} (s)")
    return result

In [ ]:
result_fgr = global_registration_by_fgr(
    src_pcd, src_feature, tgt_pcd, tgt_feature, VOXEL_SIZE
)
draw_registration_results(
    src_pcd, tgt_pcd, result_fgr.transformation, "after global registration [FGR]"
)

## 4. Local Registration (ICP)

Finally,  we demonstrate ICP, a local registration method that is capable of refining rough intial aligment from global registration methods. 

In [ ]:
def local_registration_by_icp(
    src_pcd, src_feature, tgt_pcd, tgt_feature, trans_init, voxel_size
):
    result = o3d.pipelines.registration.registration_icp(
        src_pcd,
        tgt_pcd,
        voxel_size * 0.4,
        init=trans_init,
        estimation_method=o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    )
    return result

In [ ]:
result_local = local_registration_by_icp(
    src_pcd, src_feature, tgt_pcd, tgt_feature, result_ransac.transformation, VOXEL_SIZE
)
draw_registration_results(
    src_pcd, tgt_pcd, result_local.transformation, "after local registration"
)

## 5. Multi-way Registration

This code below prepare the dataset for multi-way registration.

In [ ]:
def prepare_multiway_dataset(voxel_size):
    results = []
    dataset = o3d.data.DemoICPPointClouds()
    for p in dataset.paths:
        pcd = o3d.io.read_point_cloud(p)
        pcd_down = pcd.voxel_down_sample(voxel_size=voxel_size)
        results.append(pcd_down)
    return results

First we setup the hyperpatemeters including voxel size and two distance thresholds for hierarchical ICP.  

As can be seen in the visualization, the point clouds are provided without alignment.

In [ ]:
VOXEL_SIZE = 0.02
max_correspondence_distance_coarse = VOXEL_SIZE * 15
max_correspondence_distance_fine = VOXEL_SIZE * 1.5

pcds = prepare_multiway_dataset(voxel_size=VOXEL_SIZE)
o3d.visualization.draw_geometries(
    pcds,
    zoom=0.3412,
    front=[0.4257, -0.2125, -0.8795],
    lookat=[2.6172, 2.0475, 1.532],
    up=[-0.0694, -0.9768, 0.2024],
    window_name="before multiway registration",
)

Below two function defines the pairwise registration and multi-way registration function.

For pairwise registration, we apply ICP twice in coarse-to-fine manner to reduce the computational cost.  

In ```multiway_registration``` function, we construct the initial pose graph.  
Each node in pose graph is the input point cloud fragments, and we initialize edges using the relative transformation information predicted with ```pairwise_registration``` function. 

In [ ]:
def pairwise_registration(source, target, distance_coarse, distance_fine):
    print("Apply point-to-plane ICP")
    icp_coarse = o3d.pipelines.registration.registration_icp(
        source,
        target,
        distance_coarse,
        np.identity(4),
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    )
    icp_fine = o3d.pipelines.registration.registration_icp(
        source,
        target,
        distance_fine,
        icp_coarse.transformation,
        o3d.pipelines.registration.TransformationEstimationPointToPlane(),
    )
    transformation_icp = icp_fine.transformation
    information_icp = (
        o3d.pipelines.registration.get_information_matrix_from_point_clouds(
            source, target, distance_fine, icp_fine.transformation
        )
    )
    return transformation_icp, information_icp


def multiway_registration(pcds, distance_coarse, distance_fine):
    pose_graph = o3d.pipelines.registration.PoseGraph()
    odometry = np.identity(4)
    pose_graph.nodes.append(o3d.pipelines.registration.PoseGraphNode(odometry))
    n_pcds = len(pcds)
    for source_id in range(n_pcds):
        for target_id in range(source_id + 1, n_pcds):
            transformation_icp, information_icp = pairwise_registration(
                pcds[source_id], pcds[target_id], distance_coarse, distance_fine
            )
            print("Build o3d.pipelines.registration.PoseGraph")
            if target_id == source_id + 1:  # odometry case
                odometry = np.dot(transformation_icp, odometry)
                pose_graph.nodes.append(
                    o3d.pipelines.registration.PoseGraphNode(np.linalg.inv(odometry))
                )
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(
                        source_id,
                        target_id,
                        transformation_icp,
                        information_icp,
                        uncertain=False,
                    )
                )
            else:  # loop closure case
                pose_graph.edges.append(
                    o3d.pipelines.registration.PoseGraphEdge(
                        source_id,
                        target_id,
                        transformation_icp,
                        information_icp,
                        uncertain=True,
                    )
                )
    return pose_graph

Build initial pose graph

In [ ]:
pose_graph = multiway_registration(
    pcds, max_correspondence_distance_coarse, max_correspondence_distance_fine
)

Then we apply global optimization algorithm on the constructed pose graph to perform multiway registration.

In [ ]:
option = o3d.pipelines.registration.GlobalOptimizationOption(
    max_correspondence_distance=max_correspondence_distance_fine,
    edge_prune_threshold=0.25,
    reference_node=0,
)
o3d.pipelines.registration.global_optimization(
    pose_graph,
    o3d.pipelines.registration.GlobalOptimizationLevenbergMarquardt(),
    o3d.pipelines.registration.GlobalOptimizationConvergenceCriteria(),
    option,
)

After the pose graph is optimized, we visualize the results.

As seen in the visualization, the multiway registration algorithm of Open3D succesfully register multiple point cloud fragments.

In [ ]:
for point_id in range(len(pcds)):
    pcds[point_id].transform(pose_graph.nodes[point_id].pose)
o3d.visualization.draw_geometries(
    pcds,
    zoom=0.3412,
    front=[0.4257, -0.2125, -0.8795],
    lookat=[2.6172, 2.0475, 1.532],
    up=[-0.0694, -0.9768, 0.2024],
    window_name="after multiway registration",
)